In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

In [2]:
filenames = glob.glob('Datathon*.csv')

In [3]:
pre_dat = pd.DataFrame({})
for ind, filename in enumerate(filenames):
    print(ind)
    tmp_dat = pd.read_csv(filename)
    pre_dat = pre_dat.append(tmp_dat)    
pre_dat.columns = ['BET_ID', 'BET_TRANS_ID', 'MATCH_BET_ID', 'ACCOUNT_ID',
       'COUNTRY_OF_RESIDENCE_NAME', 'PARENT_EVENT_ID', 'EVENT_ID', 'MATCH',
       'EVENT_NAME', 'EVENT_DT', 'OFF_DT', 'BID_TYP', 'STATUS_ID',
       'PLACED_DATE', 'TAKEN_DATE', 'SETTLED_DATE', 'CANCELLED_DATE',
       'SELECTION_NAME', 'PERSISTENCE_TYPE', 'BET_PRICE', 'PRICE_TAKEN',
       'INPLAY_BET', 'BET_SIZE', 'PROFIT_LOSS']
peek_dat = pd.read_csv('DSMDatathon_samplefile.txt', sep='\t')
pre_dat = peek_dat.append(pre_dat)

0
1

/Users/feizhan/miniconda/envs/py3_env/lib/python3.4/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)
/Users/feizhan/miniconda/envs/py3_env/lib/python3.4/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)



2
3
4


In [4]:
pre_dat.loc[:, 'BID_TYP'] = pre_dat.BID_TYP.str.strip(' ')
pre_dat.loc[:, 'STATUS_ID'] = pre_dat.STATUS_ID.str.strip(' ')
# correct the profit loss
c1 = ((pre_dat.BID_TYP == 'B') & (pre_dat.PROFIT_LOSS > 0))
c2 = ((pre_dat.BID_TYP == 'B') & (pre_dat.PROFIT_LOSS < 0))
c3 = ((pre_dat.BID_TYP == 'L') & (pre_dat.PROFIT_LOSS > 0))
c4 = ((pre_dat.BID_TYP == 'L') & (pre_dat.PROFIT_LOSS < 0))
pre_dat.loc[c1.values, 'correct_profit_loss'] = pre_dat.loc[c1.values, 'BET_SIZE'] * (pre_dat.loc[c1.values, 'PRICE_TAKEN'] -1)
pre_dat.loc[c2.values, 'correct_profit_loss'] = - pre_dat.loc[c2.values, 'BET_SIZE']
pre_dat.loc[c3.values, 'correct_profit_loss'] = pre_dat.loc[c3.values, 'BET_SIZE']
pre_dat.loc[c4.values, 'correct_profit_loss'] = - pre_dat.loc[c4.values, 'BET_SIZE'] * (pre_dat.loc[c4.values, 'PRICE_TAKEN'] -1)

In [6]:
pre_dat_by_account = pre_dat.groupby('ACCOUNT_ID')

In [7]:
is_profit = pre_dat_by_account['correct_profit_loss'].sum() > 0

In [10]:
# get profitting ids and non-profitting ids
profit_ids = is_profit.loc[is_profit.values].index.tolist()
non_profit_ids = is_profit.loc[~is_profit.values].index.tolist()

In [14]:
# separate their datset
profit_id_dat = pre_dat.loc[pre_dat.ACCOUNT_ID.isin(profit_ids)]
non_profit_id_dat = pre_dat.loc[pre_dat.ACCOUNT_ID.isin(non_profit_ids)]

In [18]:
profit_lay_price = profit_id_dat.loc[(profit_id_dat.BID_TYP == 'L').values, 'PRICE_TAKEN']
non_profit_lay_price = non_profit_id_dat.loc[(non_profit_id_dat.BID_TYP == 'L').values, 'PRICE_TAKEN']

In [21]:
profit_lay_price.describe()

count    783727.000000
mean          5.667872
std          35.820739
min           1.010000
25%           1.180000
50%           1.400000
75%           1.780000
max        1000.000000
Name: PRICE_TAKEN, dtype: float64

In [22]:
non_profit_lay_price.describe()

count    625824.000000
mean          4.882091
std          34.788893
min           1.010000
25%           1.150000
50%           1.320000
75%           1.610000
max        1000.000000
Name: PRICE_TAKEN, dtype: float64

In [23]:
profit_back_price = profit_id_dat.loc[(profit_id_dat.BID_TYP == 'B').values, 'PRICE_TAKEN']
non_profit_back_price = non_profit_id_dat.loc[(non_profit_id_dat.BID_TYP == 'B').values, 'PRICE_TAKEN']

In [24]:
profit_back_price.describe()

count    792946.000000
mean          4.868061
std          32.434172
min           1.010000
25%           1.160000
50%           1.340000
75%           1.680000
max        1000.000000
Name: PRICE_TAKEN, dtype: float64

In [25]:
non_profit_back_price.describe()

count    628337.000000
mean          5.806731
std          38.550121
min           1.010000
25%           1.180000
50%           1.380000
75%           1.710000
max        1000.000000
Name: PRICE_TAKEN, dtype: float64